In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [3]:
X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))
X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())
X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [4]:
X = map(lambda x: ' '.join(x), X_train['question1'])
X1 = map(lambda x: ' '.join(x), X_train['question2'])
Y = map(lambda x: ' '.join(x), X_test['question1'])
Y1 = map(lambda x: ' '.join(x), X_test['question2'])

hw = HashingVectorizer(n_features=200).fit(X_train['question1'] + X_train['question2'] + X_test['question1'] + X_test['question2'])


In [5]:
X = pd.DataFrame(hw.transform(X).todense())
X1 = pd.DataFrame(hw.transform(X1).todense())
#del Y
#del Y1
Y = pd.DataFrame(hw.transform(Y).todense())
Y1 = pd.DataFrame(hw.transform(Y1).todense())



In [6]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X+X1+Y+Y1)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [7]:
X = pd.DataFrame(tfidf.transform(X).todense())
X1 = pd.DataFrame(tfidf.transform(X1).todense())
Y = pd.DataFrame(tfidf.transform(Y).todense())
Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

X_train = pd.concat([X_train, X], axis=1, join_axes=[X_train.index])
X_train = pd.concat([X_train, X1], axis=1, join_axes=[X_train.index])

X_test = pd.concat([X_test, Y], axis=1, join_axes=[X_test.index])
X_test = pd.concat([X_test, Y1], axis=1, join_axes=[X_test.index])

In [12]:
X_test[:2]

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.25899,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.373535,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [8]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [19]:
param = {}
param['objective'] = 'multi:softprob'
param['eval_metric'] = 'mlogloss'
param['max_depth'] = 5
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.3
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
param['num_class'] = 3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 100

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 50,
    #'max_depth': 6
    #'num_class': 3
    
}

pred_train = np.zeros((len(y_train), 2))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1

        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=20)

        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        '''
        
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])
        '''
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred)
    pred_train[ite] = ypred
    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite, 1]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/lightgbm-0.1-py3.6.egg/lightgbm/engine.py:163: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
  booster = Booster(params=params, train_set=train_set)
/home/fao3864/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:76: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y 

XGB: 0.481 +- 0.002


In [ ]:
0.481

In [22]:
pred_train = pd.DataFrame(pred_train[:, 1])
pred_train.columns = ['y']
pred_train.to_csv("stacking/lgb_1.csv", index=None)

In [33]:
submit = pd.read_csv("data/sample_submission.csv")

In [20]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
lgb.fit(X_train, y_train)
test_pred = lgb.predict_proba(X_test)

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/lightgbm-0.1-py3.6.egg/lightgbm/engine.py:163: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
  booster = Booster(params=params, train_set=train_set)
/home/fao3864/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
  app.launch_new_instance()


In [29]:
submit['is_duplicate'] = test_pred[:, 1]

In [32]:
submit.to_csv("stacking/lgb_1_test.csv", index=None)